In [149]:
import pandas as pd
import numpy as np

In [150]:
file_paths = [
    'blackhole.csv',
    'dodag.csv',
    'flooding.csv',
    'rank.csv'
]

In [151]:
# Load all CSV files into a list of DataFrames
dfs = [pd.read_csv(file) for file in file_paths]

In [152]:
# Concatenate all DataFrames into a single DataFrame
data = pd.concat(dfs,ignore_index=True)

In [153]:
# Display the first few rows
print("Dataset Preview:")
print(data.head())

Dataset Preview:
    time  source  destination  length  info  transmission_rate_per_1000_ms  \
0  0.037      39         9999     0.0   1.0                       0.000000   
1  0.037      39         9999     0.0   1.0                       0.000000   
2  0.038      39         9999     0.0   1.0                       0.671176   
3  0.045      39         9999     0.0   1.0                       0.000000   
4  0.046      39         9999     0.0   1.0                       0.000000   

   reception_rate_per_1000_ms  transmission_average_per_sec  \
0                    0.671176                      0.000000   
1                    0.649873                      0.000000   
2                    0.652361                      0.462516   
3                    0.633786                      0.000000   
4                    0.630378                      0.000000   

   reception_average_per_sec  transmission_count_per_sec  \
0                   0.499879                    0.000000   
1              

In [154]:
# Basic dataset information
print("\nDataset Info:")
print(data.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639975 entries, 0 to 1639974
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   time                                 1639975 non-null  float64
 1   source                               1639975 non-null  int64  
 2   destination                          1639975 non-null  int64  
 3   length                               1639975 non-null  float64
 4   info                                 1639975 non-null  float64
 5   transmission_rate_per_1000_ms        1639975 non-null  float64
 6   reception_rate_per_1000_ms           1639975 non-null  float64
 7   transmission_average_per_sec         1639975 non-null  float64
 8   reception_average_per_sec            1639975 non-null  float64
 9   transmission_count_per_sec           1639975 non-null  float64
 10  reception_count_per_sec              1639975 non-nu

In [155]:
# Check for missing values
print("\nMissing Values (Before Handling):")
print(data.isnull().sum())


Missing Values (Before Handling):
time                                   0
source                                 0
destination                            0
length                                 0
info                                   0
transmission_rate_per_1000_ms          0
reception_rate_per_1000_ms             0
transmission_average_per_sec           0
reception_average_per_sec              0
transmission_count_per_sec             0
reception_count_per_sec                0
transmission_total_duration_per_sec    0
reception_total_duration_per_sec       0
dao                                    0
dis                                    0
dio                                    0
category                               0
label                                  0
dtype: int64


In [156]:
data.nunique().sort_values()

label                                       2
category                                    5
info                                       10
length                                     13
destination                               101
source                                    101
dis                                       206
dao                                       233
dio                                       383
transmission_count_per_sec                467
transmission_rate_per_1000_ms             470
reception_total_duration_per_sec          575
reception_rate_per_1000_ms               1248
reception_count_per_sec                  1253
transmission_total_duration_per_sec      1634
reception_average_per_sec                9369
transmission_average_per_sec            21764
time                                   725703
dtype: int64

In [157]:
print(data['info'].sort_values().unique())

[0.         0.41111834 0.52029776 0.55440486 0.57018941 0.57241282
 0.57289431 0.58018146 0.58259798 1.        ]


In [158]:
import tensorflow as tf
from tensorflow.keras import layers

In [159]:
source_lookup = tf.keras.layers.IntegerLookup(output_mode='int')
destination_lookup = tf.keras.layers.IntegerLookup(output_mode='int')
info_lookup = tf.keras.layers.StringLookup(output_mode='int')
length_lookup = tf.keras.layers.StringLookup(output_mode='int')

source_lookup.adapt(data['source'])
destination_lookup.adapt(data['destination'])
info_lookup.adapt(data['info'].astype(str))
length_lookup.adapt(data['length'].astype(str))


In [160]:
source_embedding_layer = layers.Embedding(input_dim=source_lookup.vocabulary_size(), output_dim= 32)
source_input = tf.keras.Input(shape=(1,), dtype=tf.int64, name='source')
source_index = source_lookup(source_input)
source_embedding = source_embedding_layer(source_index)

destination_embedding_layer = layers.Embedding(input_dim=destination_lookup.vocabulary_size(), output_dim= 32)
destination_input = tf.keras.Input(shape=(1,), dtype=tf.int64, name= 'destination')
destination_index = destination_lookup(destination_input)
destination_embedding = destination_embedding_layer(source_index)

info_embedding_layer = layers.Embedding(input_dim=info_lookup.vocabulary_size(), output_dim= 32)
info_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='info')
info_index = info_lookup(info_input)
info_embedding = info_embedding_layer(info_index)

length_embedding_layer = layers.Embedding(input_dim=length_lookup.vocabulary_size(), output_dim= 32)
length_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='length')
length_index = length_lookup(length_input)
length_embedding = length_embedding_layer(length_index)

In [161]:
# Convert a value to index in vocabulary
# value = tf.constant(39)
# print(type(value))
# index = source_lookup(value)

# # Get corresponding Embedding vector  
# embedding_vector = source_embedding_layer(index)
# print(embedding_vector)

In [162]:
# Convert a value to index in vocabulary
# value = tf.constant("0.58018146")
# print(type(value))
# index = info_lookup(value)

# # Get corresponding Embedding vector  
# embedding_vector = info_embedding_layer(index)
# print(embedding_vector)

In [163]:
# Numerical -> Vector

"""
    DIS
"""
dis_norm_layer = tf.keras.layers.Normalization()
dis_norm_layer.adapt(data['dis'].values.reshape(-1, 1))

dis_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dis')
dis_norm = dis_norm_layer(dis_input)
dis_vector = layers.Dense(32)(dis_norm)
dis_vector = layers.Reshape((1, 32))(dis_vector)

"""
    DAO
"""
dao_norm_layer = tf.keras.layers.Normalization()
dao_norm_layer.adapt(data['dao'].values.reshape(-1, 1))

dao_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dao')
dao_norm = dao_norm_layer(dao_input)
dao_vector = layers.Dense(32)(dao_norm)
dao_vector = layers.Reshape((1, 32))(dao_vector)

"""
    DIO feature
"""
dio_norm_layer = tf.keras.layers.Normalization()
dio_norm_layer.adapt(data['dio'].values.reshape(-1, 1))

dio_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dio')
dio_norm = dio_norm_layer(dio_input)
dio_vector = layers.Dense(32)(dio_norm)
dio_vector = layers.Reshape((1, 32))(dio_vector)

"""
    transmission_count_per_sec feature
"""
transmission_count_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_count_per_sec_norm_layer.adapt(data['transmission_count_per_sec'].values.reshape(-1, 1))


transmission_count_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_count_per_sec')
transmission_count_per_sec_norm = transmission_count_per_sec_norm_layer(transmission_count_per_sec_input)
transmission_count_per_sec_vector = layers.Dense(32)(transmission_count_per_sec_norm)
transmission_count_per_sec_vector = layers.Reshape((1, 32))(transmission_count_per_sec_vector)

"""
    transmission_rate_per_1000_ms feature
"""

transmission_rate_per_1000_ms_norm_layer = tf.keras.layers.Normalization()
transmission_rate_per_1000_ms_norm_layer.adapt(data['transmission_rate_per_1000_ms'].values.reshape(-1, 1))

transmission_rate_per_1000_ms_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_rate_per_1000_ms')
transmission_rate_per_1000_ms_norm = transmission_rate_per_1000_ms_norm_layer(transmission_rate_per_1000_ms_input)
transmission_rate_per_1000_ms_vector = layers.Dense(32)(transmission_rate_per_1000_ms_norm)
transmission_rate_per_1000_ms_vector = layers.Reshape((1, 32))(transmission_rate_per_1000_ms_vector)

"""
    time_input feature
"""
time_input_norm_layer = tf.keras.layers.Normalization()
time_input_norm_layer.adapt(data['time'].values.reshape(-1, 1))

time_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='time')
time_norm = time_input_norm_layer(time_input)
time_vector = layers.Dense(32)(time_norm)
time_vector = layers.Reshape((1, 32))(time_vector)

"""
    reception_total_duration_per_sec feature 
"""
reception_total_duration_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_total_duration_per_sec_norm_layer.adapt(data['reception_total_duration_per_sec'].values.reshape(-1, 1))

reception_total_duration_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_total_duration_per_sec')
reception_total_duration_per_sec_norm = reception_total_duration_per_sec_norm_layer(reception_total_duration_per_sec_input)
reception_total_duration_per_sec_vector = layers.Dense(32)(reception_total_duration_per_sec_norm)
reception_total_duration_per_sec_vector = layers.Reshape((1, 32))(reception_total_duration_per_sec_vector)

"""
    reception_rate_per_1000_ms feature 
"""
reception_rate_per_1000_ms_norm_layer = tf.keras.layers.Normalization()
reception_rate_per_1000_ms_norm_layer.adapt(data['reception_rate_per_1000_ms'].values.reshape(-1, 1))

reception_rate_per_1000_ms_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_rate_per_1000_ms')
reception_rate_per_1000_ms_norm = reception_rate_per_1000_ms_norm_layer(reception_rate_per_1000_ms_input)
reception_rate_per_1000_ms_vector = layers.Dense(32)(reception_rate_per_1000_ms_norm)
reception_rate_per_1000_ms_vector = layers.Reshape((1, 32))(reception_rate_per_1000_ms_vector)

"""
    reception_count_per_sec feature
"""
reception_count_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_count_per_sec_norm_layer.adapt(data['reception_count_per_sec'].values.reshape(-1,1))

reception_count_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_count_per_sec')
reception_count_per_sec_norm = reception_count_per_sec_norm_layer(reception_count_per_sec_input)
reception_count_per_sec_vector = layers.Dense(32)(reception_count_per_sec_norm)
reception_count_per_sec_vector = layers.Reshape((1, 32))(reception_count_per_sec_vector)

"""
    transmission_total_duration_per_sec feature
"""
transmission_total_duration_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_total_duration_per_sec_norm_layer.adapt(data['transmission_total_duration_per_sec'].values.reshape(-1,1))

transmission_total_duration_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_total_duration_per_sec')
transmission_total_duration_per_sec_norm = transmission_total_duration_per_sec_norm_layer(transmission_total_duration_per_sec_input)
transmission_total_duration_per_sec_vector = layers.Dense(32)(transmission_total_duration_per_sec_norm)
transmission_total_duration_per_sec_vector = layers.Reshape((1, 32))(transmission_total_duration_per_sec_vector)

"""
    reception_average_per_sec feature
"""
reception_average_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_average_per_sec_norm_layer.adapt(data['reception_average_per_sec'].values.reshape(-1,1))

reception_average_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_average_per_sec')
reception_average_per_sec_norm = reception_average_per_sec_norm_layer(reception_average_per_sec_input)
reception_average_per_sec_vector = layers.Dense(32)(reception_average_per_sec_norm)
reception_average_per_sec_vector = layers.Reshape((1, 32))(reception_average_per_sec_vector)

"""
    transmission_average_per_sec feature
"""
transmission_average_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_average_per_sec_norm_layer.adapt(data['transmission_average_per_sec'].values.reshape(-1,1))

transmission_average_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_average_per_sec')
transmission_average_per_sec_norm = transmission_average_per_sec_norm_layer(transmission_average_per_sec_input)
transmission_average_per_sec_vector = layers.Dense(32)(transmission_average_per_sec_norm)
transmission_average_per_sec_vector = layers.Reshape((1, 32))(transmission_average_per_sec_vector)


In [164]:
print(dao_vector.shape)

(None, 1, 32)


In [165]:
concatenated = layers.Concatenate(axis=1)([time_vector, 
                                           source_embedding, 
                                           destination_embedding, 
                                           length_embedding, 
                                           info_embedding,
                                           transmission_rate_per_1000_ms_vector,
                                           reception_rate_per_1000_ms_vector,
                                           transmission_average_per_sec_vector,
                                           reception_average_per_sec_vector,
                                           transmission_count_per_sec_vector,
                                           reception_count_per_sec_vector,
                                           transmission_total_duration_per_sec_vector,
                                           reception_total_duration_per_sec_vector,
                                           dao_vector,
                                           dis_vector,
                                           dio_vector])
print(concatenated.shape)

(None, 16, 32)


In [166]:
def transformer_block(x, num_heads=4, ff_dim=64, dropout=0.1):
    # Multi-head attention
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                      key_dim=x.shape[-1])(x, x)
    attn_output = tf.keras.layers.Dropout(dropout)(attn_output)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed-forward layer
    ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(ff_dim, activation="relu"),
        tf.keras.layers.Dense(x.shape[-1]),
    ])
    ffn_output = ffn(out1)
    ffn_output = tf.keras.layers.Dropout(dropout)(ffn_output)
    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

In [167]:
# Giả sử 'concatenated' có shape (None, 16, 32)
x = transformer_block(concatenated)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = layers.Dense(1, activation='sigmoid')(x)

In [168]:
inputs=[    
    time_input, 
    source_input, 
    destination_input, 
    length_input, 
    info_input,
    transmission_rate_per_1000_ms_input,
    reception_rate_per_1000_ms_input,
    transmission_average_per_sec_input,
    reception_average_per_sec_input,
    transmission_count_per_sec_input,
    reception_count_per_sec_input,
    transmission_total_duration_per_sec_input,
    reception_total_duration_per_sec_input,
    dao_input,
    dis_input,
    dio_input]
print(len(inputs))
model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


16


In [169]:
model.summary()

Model: "functional_24"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ time (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_rate_… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_rate_per… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_avera… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_average_… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_count… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_count_pe… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_total… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_total_du… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dao (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dis (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dio (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_65    │ (None, 1)         │          3 │ time[0][0]        │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ source (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ length (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ info (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_64    │ (None, 1)         │          3 │ transmission_rat… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_67    │ (None, 1)         │          3 │ reception_rate_p… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_71    │ (None, 1)         │          3 │ transmission_ave… │
│ (Normalization)     │                   │            │                 

 Total params: 50,405 (196.94 KB)

 Trainable params: 50,369 (196.75 KB)

 Non-trainable params: 36 (192.00 B)

In [170]:
X = data.drop(columns='label')
X = X.drop(columns='category')
y = data['label'].values

In [180]:
x = {
    col: (
        X[col].values.astype('<U32').reshape(-1, 1) if col in ['info', 'length']
        else X[col].values.reshape(-1, 1)
    )
    for col in X.columns
}

In [184]:
for col in ['info', 'length']:
    x[col] = tf.constant(x[col].flatten(), dtype=tf.string)
    x[col] = tf.reshape(x[col], (-1, 1))

In [182]:
for col in x.keys():
    print(tf.constant(x[col]).dtype)

<dtype: 'float64'>
<dtype: 'int64'>
<dtype: 'int64'>
<dtype: 'string'>
<dtype: 'string'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>
<dtype: 'float64'>


In [181]:
print(type(x['length'][0][0])) 

<class 'numpy.str_'>


In [183]:
print(x['length'].dtype)

<U32


In [175]:
x.keys()

dict_keys(['time', 'source', 'destination', 'length', 'info', 'transmission_rate_per_1000_ms', 'reception_rate_per_1000_ms', 'transmission_average_per_sec', 'reception_average_per_sec', 'transmission_count_per_sec', 'reception_count_per_sec', 'transmission_total_duration_per_sec', 'reception_total_duration_per_sec', 'dao', 'dis', 'dio'])

In [176]:
type(x['info'][0])

numpy.ndarray

In [185]:
model.fit(
    x=x,
    y=y,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
 6141/41000 ━━━━━━━━━━━━━━━━━━━━ 27:43 48ms/step - accuracy: 0.9136 - loss: 0.2058

KeyboardInterrupt: 

In [ ]:
# GlobalAveragePooling hoặc GlobalMaxPooling
x = tf.keras.layers.GlobalAveragePooling1D()(x)  # (None, 32)

# Dense classifier
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Binary classification


In [ ]:
attention_output = layers.MultiHeadAttention(
    num_heads=4,         # Số lượng "head" attention
    key_dim=32           # Độ dài vector key/query — thường là embedding_dim / num_heads
)(query=norm_concatenated, key=norm_concatenated, value=norm_concatenated)

In [ ]:
input_layernorm = layers.Add()([norm_concatenated, attention_output])
output_layernorm = layers.LayerNormalization(epsilon=1e-6)(input_layernorm)

In [ ]:
ffn = layers.Dense(128, activation='relu')(output_layernorm)
ffn = layers.Dense(32)(ffn)

# Residual + LayerNorm
x = layers.Add()([output_layernorm, ffn])
x = layers.LayerNormalization(epsilon=1e-6)(x)

In [ ]:
x = layers.GlobalAveragePooling1D()(x)  # Shape: (batch, 32)
output = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = tf.keras.Model(inputs=[source_input, dis_input, ...], outputs=output)